In [ ]:
! pip install azure-search-documents --pre
! pip install python-dotenv

In [24]:
# Import required libraries  
import os  
import json  
import openai
import keys
from dotenv import load_dotenv  
from tenacity import retry, wait_random_exponential, stop_after_attempt  
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient  
from azure.search.documents.indexes import SearchIndexClient  
from azure.search.documents.models import Vector  
from azure.search.documents.indexes.models import (  
    SearchIndex,  
    SearchField,  
    SearchFieldDataType,  
    SimpleField,  
    SearchableField,  
    SearchIndex,  
    SemanticConfiguration,  
    PrioritizedFields,  
    SemanticField,  
    SearchField,  
    SemanticSettings,  
    VectorSearch,  
    HnswVectorSearchAlgorithmConfiguration,  
)

import pandas as pd
  
# Configure environment variables  
load_dotenv()  
service_endpoint = "https://aimlexplorationsearch.search.windows.net"
index_name = "rsaf-cognitive-search"
key = "hwioAbEseT8hs3Pv98fREcnMpoqPQnp9FdR0OwoPpmAzSeCG2vMW"

# key = "wGvVdInupyqUH2YraI1mpZmviDLXjh3v3i2zZetKAnAzSeCZGrhS"
# service_endpoint = "https://rsaf-cognitive-search-service-us.search.windows.net"

openai.api_type = "azure"
openai.api_key = "0e576c031e5446e19cc6d866ad3d9f3f"
openai.api_base = "https://raid-ses-openai.openai.azure.com/"
openai.api_version = "2023-05-15"
credential = AzureKeyCredential(key)

In [25]:
client_index = SearchIndexClient(endpoint=service_endpoint, credential=credential)

In [32]:
client_index.delete_index("rsaf-cognitive-search")

In [33]:
for i in client_index.list_indexes():
    print(i.name)

rsaf-cognitive-search


In [ ]:
from azure.search.documents.models import (
    QueryAnswerType,
    QueryCaptionType,
    QueryLanguage,
    QueryType  
)

search_client = SearchClient(endpoint=service_endpoint, index_name="chat-demo", credential=credential)

all_docs = search_client.search(search_text="autorotation", query_type=QueryType.SEMANTIC, query_language=QueryLanguage.EN_US, semantic_configuration_name='default',top=6)

pd.DataFrame(all_docs)

In [58]:
search_client = SearchClient(endpoint=service_endpoint, index_name="rsaf-cognitive-search", credential=credential)

all_docs = search_client.search(search_text="*", top=6)

In [59]:
for i in all_docs:
    text = "Filename: " + i["filename"] + "\n" + "Page Number: " + str(i["page"]) + "\n" + "Content: " + i["content"] + "\n-----------------------"
    print(text)

Filename: EEM FINAL caa201221-1-50.pdf
Page Number: 6
Content: OFFICIAL (CLOSED)
PART A
CHAPTER 1
INTRODUCTION AND FAMILIARISATION
INTRODUCTION
1.1
This chapter orientates crew, with a focus on RWC trainees, to the
squadron's administrative processes, preparation for flight training on the
EC120B in Singapore and common practices and terms used in helicopter
operations. In addition, an introduction to the EC120B helicopter and its unique
characteristics is provided (See Annex A).
1.2
EEM. The EC employment manual serves to provide RWC trainees, QHIs
and all crew operating the EC120B details of the conduct of manoeuvres along
with linking the application to basic helicopter principles of flight and providing
the context of the manoeuvre to application in helicopter operations. Each chapter
employs the "Who, What, When, Where, Why and How" framework in
structuring content. The structure and common terms used in the EEM are as
follows:
Table 1-1: Generic EEM Structure
Sub-Topic
5Ws and 1H

In [7]:
results = search_client.search(search_text="Advanced Transitions confined spaces", select="content", top=3)

### Vector Search

In [12]:
def generate_embeddings(text):
    response = openai.Embedding.create(
        input=text, engine="swiftfaq-ada002")
    embeddings = response['data'][0]['embedding']
    return embeddings

In [30]:
search_client = SearchClient(endpoint=service_endpoint, index_name="chat-demo", credential=credential)

pd.DataFrame(search_client.search("autorotation", top=3))

,embedding,id,li_jsonMetadata,li_doc_id,page_label,content,@search.score,@search.reranker_score,@search.highlights,@search.captions
0,"[-0.021303589, -0.008744429, -0.020742599, -0....",21c6d4c3-6106-4f3d-b628-73e129507b7c,"{""page_label"": ""111"", ""file_name"": ""EEM FINAL ...",2da13a2e-fbc3-4d15-b8fa-6e1517074460,111,OFFICIAL (CLOSED) \n11-1 \nOFFICIAL (CLOSED)...,3.408726,None,None,None
1,"[-0.017312882, -0.0033271702, -0.012544677, -0...",2f71f668-85dc-4155-9b57-104767b0c28b,"{""page_label"": ""42"", ""file_name"": ""EEM FINAL c...",87546a10-0794-45fa-a8cf-46919ebe14ce,42,OFFICIAL (CLOSED) \n5-1 \nOFFICIAL (CLOSED) ...,3.335187,None,None,None
2,"[-0.022873871, -0.013425363, -0.0023482218, -0...",2282ddc7-7b38-4f25-815e-dc8d3f323e1d,"{""page_label"": ""116"", ""file_name"": ""EEM FINAL ...",bd14dd00-8091-4885-9421-86003faa860b,116,OFFICIAL (CLOSED) \n11-6 \nOFFICIAL (CLOSED)...,3.253138,None,None,None


In [17]:
query = "What is autorotation?"

search_client = SearchClient(endpoint=service_endpoint, index_name="rsaf-cognitive-search", credential=credential)

vector = VectorizableTextQuery(value=generate_embeddings(query), k=3, fields="embedding")
results = search_client.search(search_text=None, vectors=[vector], filter="", top=3)

value is not a known attribute of class <class 'azure.search.documents._generated.models._models_py3.RawVectorQuery'> and will be ignored
